# Experiment 1

In [ ]:
from transformer import Transformer
from dataset.scan_dataset import ScanDataset, ExperimentType
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)
# device = "cpu"
max_len = 128
train_dataset = ScanDataset(
    ExperimentType.E1_TRAIN,
    in_seq_len=max_len,
    out_seq_len=max_len + 20,
    device=device,
)
test_dataset = ScanDataset(
    ExperimentType.E1_TEST,
    vocab=train_dataset.vocab,
    in_seq_len=max_len,
    out_seq_len=max_len + 20,
    device=device,
)

model = Transformer(
    src_vocab_size=len(train_dataset.vocab),
    tgt_vocab_size=len(train_dataset.vocab),
    src_pad_idx=train_dataset.vocab.pad_idx,
    tgt_pad_idx=train_dataset.vocab.pad_idx,
    dropout=0.05,
    emb_dim=128,
    num_layers=1,
    num_heads=8,
    forward_dim=512,
    max_len=max_len + 20,
)

In [2]:
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from torch.nn import utils
import torch

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)
grad_clip = 1.0
criterion = CrossEntropyLoss(ignore_index=train_dataset.vocab.pad_idx)
optimizer = Adam(
    model.parameters(),
    lr=7e-4,
    weight_decay=0.00001,
)

from tqdm import tqdm

model.to(device)

for epoch in range(70):
    losses = []
    for step, batch in enumerate(tqdm(train_loader)):
        inputs, decoder_inputs, target_label_indices = batch

        optimizer.zero_grad()
        out = model(inputs, decoder_inputs)
        loss = criterion(out.permute(0, 2, 1), target_label_indices)
        loss.backward()
        utils.clip_grad_norm_(model.parameters(), grad_clip)
        optimizer.step()

        losses.append(loss.item())
        if step % 100 == 0:
            print(f"Epoch {epoch} Loss: {sum(losses) / len(losses)}")

  6%|▌         | 16/262 [00:00<00:03, 70.25it/s]

Epoch 0 Loss: 3.1776909828186035


 48%|████▊     | 127/262 [00:00<00:00, 149.98it/s]

Epoch 0 Loss: 0.9928240339354714


 85%|████████▌ | 223/262 [00:01<00:00, 156.77it/s]

...

 45%|████▍     | 117/262 [00:00<00:00, 160.82it/s]

Epoch 69 Loss: 0.003602335573719252


 84%|████████▎ | 219/262 [00:01<00:00, 160.87it/s]

Epoch 69 Loss: 0.0022714917191949476


100%|██████████| 262/262 [00:01<00:00, 161.25it/s]


In [ ]:
from evaluate import evaluate_model_batchwise
evals = evaluate_model_batchwise(model, test_loader, train_dataset.vocab, device=device)
evals

cuda
Generated: ['I_TURN_RIGHT', 'I_TURN_RIGHT', 'I_TURN_RIGHT', 'I_TURN_RIGHT', 'I_TURN_RIGHT']
Target: ['I_TURN_RIGHT', 'I_TURN_RIGHT', 'I_TURN_RIGHT', 'I_TURN_RIGHT', 'I_TURN_RIGHT', 'I_TURN_RIGHT', 'I_TURN_RIGHT']
4181 4182


0.9997608799617408